# Memory management
This workbook show how memory management work in CPython. It can differ with other Pyhton implementation.

In [2]:
import sys

## I. Reference count
In Python, the garbage collector deletes an object when there is no more reference to it. When there is no more reference to an object the `__del__` method is called. 

The function `sys.getcountref` returns the reference count (plus one because the object is passed as argument to the function) an object. 

In [4]:
class C:
    def __del__(self):
        print(f"{self} is deleted")
        del self
    pass

In [5]:
c = C()
sys.getrefcount(c) # 2 because there is one ref associated to the argument

2

In [6]:
d = c
sys.getrefcount(c)

3

In [7]:
del d

In [8]:
del c # once reference count is down zero the object is deleted

<__main__.C object at 0x000001FD1114B080> is deleted


The object is deleted when the reference count drops down to zero.

### a. Actions that make the reference count drop

**variable reassignement**

In [12]:
l = C()

In [13]:
l = 2

<__main__.C object at 0x000001FD12078CE0> is deleted


**changing scope**

In [15]:
def func():
    l = C()

In [16]:
func()

<__main__.C object at 0x000001FD1379A5A0> is deleted


### b. Reference count with weak ref
Using weakref does not increase the ref count to an object.

In [18]:
import weakref

In [19]:
c = C()
sys.getrefcount(c)

2

In [20]:
d = weakref.ref(c)
sys.getrefcount(c)

2

In [21]:
del c # ref is not increased

<__main__.C object at 0x000001FD1114B080> is deleted


## II. Memory Management and Pitfalls
When storing objects into a container like a list or a dictionary, the reference count is increased. Even though the object is deleted it is still stored in the dictionary.

In [23]:
class A:
    def __del__(self):
        print(f"deleting {self}")
        del self

### a. using standard reference

In [25]:
my_dict = {}
a = A()
my_dict = {'a': a}

In [26]:
del a

In [27]:
my_dict

{'a': <__main__.A at 0x1fd1114b080>}

In [28]:
del my_dict['a']

deleting <__main__.A object at 0x000001FD1114B080>


### b. using weak references

In [30]:
my_dict = {}
k = A()

In [31]:
my_dict = {'k': weakref.ref(k)}

In [32]:
del k

deleting <__main__.A object at 0x000001FD11BC6480>


In [33]:
my_dict['k']

<weakref at 0x000001FD137ECC70; dead>